In [ ]:
#| default_exp analysing

In [ ]:
#| export

import json
import escher
import numpy as np
import pandas as pd
import tol_colors as tc
from escher import Builder
from matplotlib.colors import rgb2hex
from mmon_gcm.buildingediting import check_number_of_models

ImportError: cannot import name 'soft_unicode' from 'markupsafe' (/home/nls119/lib/miniconda3/envs/mmon-gcm/lib/python3.10/site-packages/markupsafe/__init__.py)

In [ ]:
#| export


def get_phase_times(model):
    return np.arange(1, check_number_of_models(model) + 1)

In [ ]:
#| export


def get_phase_length(model, phase):
    return round(
        -1
        / model.reactions.get_by_id(f"Cl_c_gc_Linker_{phase}").get_coefficient(
            f"Cl_c_gc_{phase}"
        ),
        1,
    )

In [ ]:
#| export


def get_phase_lengths(model):
    return [get_phase_length(model, phase) for phase in get_phase_times(model)]

In [ ]:
#| export


def adjust_for_phases(series_object, model):
    phase = series_object.name[1]
    phase_length = get_phase_length(model, phase)
    return series_object * phase_length

In [ ]:
#| export


def get_only_gc_opening(df):

    reactions_list = df.index
    gc_opening_reactions = [
        reaction
        for reaction in reactions_list
        if "_gc_2" in reaction
        or "_gc_Linker_2" in reaction
        and "pseudo" not in reaction
        and "EX_" not in reaction
    ]

    filtered_df = df.loc[gc_opening_reactions]

    return filtered_df

In [ ]:
#| export


def get_pearson(df):
    pearson_array = np.corrcoef(df.values)

    pearson_df = pd.DataFrame(pearson_array, index=df.index, columns=df.index)

    return pearson_df

In [ ]:
#| export


def get_solution_dfs(
    light, atpase_constrained, starch_knockout, outputs_path, cutoff_flux=1 * 10 ** -8
):

    if atpase_constrained == False:
        atpase = "unconstrained"
    else:
        atpase = "constrained"

    if starch_knockout == False:
        starch = "wt"
    else:
        starch = "ko"

    starch_translation_dict = {"starchko": "ko", "ko": "starchko", "wt": "wt"}

    alternative_modes_df = pd.read_csv(
        outputs_path + f"/alternative_weighting/solution_{light}_{atpase}_{starch}.csv",
        index_col=0,
    ).transpose()

    pfba_df = pd.read_csv(
        outputs_path
        + f"/model_solutions/{light}_{atpase}_{starch_translation_dict[starch]}.csv",
        index_col=0,
    ).loc[:, "fluxes"]

    averages_df = alternative_modes_df.mean(axis=1)

    # ignore fluxes below the cutoff_flux
    for df in [alternative_modes_df, pfba_df, averages_df]:
        df.mask(abs(df) < cutoff_flux, 0, inplace=True)

    alternative_modes_df, pfba_df, averages_df = [
        get_only_gc_opening(df) for df in [alternative_modes_df, pfba_df, averages_df]
    ]

    return alternative_modes_df, pfba_df, averages_df

In [ ]:
#| export


def check_for_missing(reaction_list, map_path):

    with open(map_path, "r") as read_file:
        map_json = json.load(read_file)

    map_rxns = [reaction["bigg_id"] for reaction in map_json[1]["reactions"].values()]

    missing_reactions = [
        reaction for reaction in reaction_list if reaction not in map_rxns
    ]

    if len(missing_reactions) != 0:
        warnings.warn("There are some reactions that aren't in the map")
        return missing_reactions

In [ ]:
#| export


def get_escher_map(
    model_solution, map="inputs/map.json", json_model="models/4_stage_GC.json", **kwargs
):
    
    builder = Builder(model_json=json_model, **kwargs)
    builder.map_json = map
    builder.reaction_data = model_solution
    builder.scroll_behavior = "zoom"
    builder.reaction_styles = ["color", "size", "text"]

    builder.reaction_no_data_color = "#DDDDDD"
    builder.reaction_no_data_size = 5

    return builder

In [ ]:
#| export


def get_difference_biggest_to_smallest(row):

    row = abs(row)

    return pd.Series(
        [(row[1] - row[0]) / row.max() * 100, row.idxmax()],
        index=["% Difference", "Larger"],
    )

In [ ]:
#| export


def replace_with_1_0(value, threshold):
    if abs(value) < threshold:
        return 0
    # elif value<threshold*-1:
    #    return -1
    else:
        return round(value, 2)

In [ ]:
#| export


def get_adjacency(df, threshold):
    np.fill_diagonal(df.values, 0)
    df = df.applymap(replace_with_1_0, threshold=threshold)
    return df

In [ ]:
#| export


def get_percentage_difference_df(pfba_df, averages_df):

    difference_df = pd.DataFrame(pfba_df - averages_df, columns=["Difference"])
    combined_df = pd.concat([pfba_df, averages_df], axis=1, keys=["pFBA", "Average"])
    combined_df = combined_df.loc[abs(difference_df.Difference) > 10 ** -8]
    percentage_difference_df = combined_df.apply(
        get_difference_biggest_to_smallest, axis=1
    )
    percentage_difference_df["% Difference"] = percentage_difference_df[
        "% Difference"
    ].round()

    return percentage_difference_df

In [ ]:
#| export


def get_difference_map(percentage_difference_df, map_path, json_model):

    reaction_scale_different = [
        {"type": "value", "value": -100, "color": "#2166AC", "size": 20},
        {"type": "value", "value": 0, "color": "#F7F7F7", "size": 20},
        {"type": "value", "value": 100, "color": "#B2182B", "size": 20},
    ]

    missing = check_for_missing(
        percentage_difference_df.index, map_path,
    )
    map = get_escher_map(
        percentage_difference_df.loc[:, "% Difference"],
        reaction_scale=reaction_scale_different,
        map=map_path,
        json_model=json_model,
    )

    return map

In [ ]:
#| export


def get_pfba_pathways_map(subgraph_list, map_path, json_model):

    colours = [
        rgb2hex(tc.tol_cmap("rainbow_discrete", lut=len(subgraph_list))(i))
        for i in np.linspace(0, 1, num=len(subgraph_list))
    ]

    reaction_scale = [
        {"type": "value", "value": i, "color": colour, "size": 30}
        for i, colour in enumerate(colours)
    ]

    reaction_values = {}

    for i, subgraph in enumerate(subgraph_list):
        for node in subgraph.nodes:
            reaction_values[node] = i

    map = get_escher_map(
        reaction_values,
        reaction_scale=reaction_scale,
        map=map_path,
        json_model=json_model,
    )

    return map